In [ ]:
#install dependencies 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits import mplot3d

from sklearn.linear_model import LinearRegression

In [ ]:
#load dataset
ER = pd.read_csv('ER Wait Time Dataset.csv')